## В этом домашнем задании вы сделаете первые шаги в мире линейной бинарной классификации!

In [1]:
import pandas as pd
import numpy as np
import warnings

#### Задание 1

Мы будем работать с данными **Microsoft Malware Detection**

Таргетом будет последний столбец `HasDetection`, который принимает значения $\{0,\, 1\}$ в случае отсутствия или наличия вируса на компьютере соответственно. Признаками будут выступать всевозможные характеристики компьютера.

In [2]:
data = pd.read_csv('train.csv')

C:\Users\e.tarasov\AppData\Local\Temp\ipykernel_1624\2815211952.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('train.csv')


In [3]:
data.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,...,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,...,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,...,52682.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,...,20050.0,0,NaN,0.0,0,0,0.0,0.0,3.0,1
4,000014a5f00daa18e76b81417eeb99fc,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1379.0,0,7.0,0,NaN,53447.0,...,19844.0,0,0.0,0.0,0,0,0.0,0.0,1.0,1


In [4]:
for col in data.columns:
    if len(data[col].unique()) == 1:
        print(col)    

IsBeta


Удалите константные признаки и признаки `ProductName` `MachineIdentifier`

In [5]:
### Your code is here
data = data.drop(columns=['ProductName', 'MachineIdentifier', 'IsBeta'])

Посмотрите на соотношение классов в таргете. Все ли хорошо?

In [6]:
print(sum(data['HasDetections'] == 1), '- positive class,')
print(sum(data['HasDetections'] == 0), '- negative class')

100060 - positive class,
99940 - negative class


Ответьте на вопрос: почему с вашей точки зрения важно иметь представление о балансе классов в ваших данных?

Избавьтесь от пропусков в данных! 

Новый для нас прием: признаки с более чем половиной пропусков следует удалить.

Согласитесь, если в вашей колонке среди 100 объектов всего лишь у 2 есть какое-то непропущенное значение, странно все остальные заполнять средним от этих двух чисел. Такие "редкие" признаки лучше вообще опустить!


В категориальных колонках заменим отсутствующую категорию просто некоторой новой и назовем ее `NaN`.

А в числовых, ради разнообразия, заполним пропуски медианным значением.

In [7]:
for col in data.columns:
    print(col, data[col].dtype)

EngineVersion object
AppVersion object
AvSigVersion object
RtpStateBitfield float64
IsSxsPassiveMode int64
DefaultBrowsersIdentifier float64
AVProductStatesIdentifier float64
AVProductsInstalled float64
AVProductsEnabled float64
HasTpm int64
CountryIdentifier int64
CityIdentifier float64
OrganizationIdentifier float64
GeoNameIdentifier float64
LocaleEnglishNameIdentifier int64
Platform object
Processor object
OsVer object
OsBuild int64
OsSuite int64
OsPlatformSubRelease object
OsBuildLab object
SkuEdition object
IsProtected float64
AutoSampleOptIn int64
PuaMode object
SMode float64
IeVerIdentifier float64
SmartScreen object
Firewall float64
UacLuaenable float64
Census_MDC2FormFactor object
Census_DeviceFamily object
Census_OEMNameIdentifier float64
Census_OEMModelIdentifier float64
Census_ProcessorCoreCount float64
Census_ProcessorManufacturerIdentifier float64
Census_ProcessorModelIdentifier float64
Census_ProcessorClass object
Census_PrimaryDiskTotalCapacity float64
Census_PrimaryDis

In [8]:
### Your code is here
many_nans_cols = []
for col in data.columns:
    if data[col].isna().sum() / data.shape[0] > 0.5:
        many_nans_cols.append(col)
data = data.drop(columns=many_nans_cols)

for col in data.columns:
    if data[col].dtype == 'object':
        data[col] = data[col].fillna("NaN")
        
for col in data.columns:
    if data[col].dtype != 'object':
        median_value = data[col].median()
        data[col] = data[col].fillna(median_value)

Создайте копию полученного датафрейма и положите ее в переменную data_2. Понадобится в следующих заданиях.

In [9]:
data_2 = data.copy()
data.shape

(200000, 73)

Так же поработаем над всеми категориальными колонкам перед запуском непосредственно моделей.

Провернем самый базовый и наглый метод - несмотря на количество уникальных значений в каждой категории, просто применим ко всей категориальной части датасета `OneHotEncoding`

In [10]:
### Your code is here
from tqdm import tqdm

total = 0
for col in data.columns:
    if data[col].dtype == 'object':
        total += len(data[col].unique())

from sklearn.preprocessing import OneHotEncoder

# data_2 = data_2.copy()
cat_cols = [col for col in data.columns if data[col].dtype == 'object']
print('Started cat_cols')
for col in tqdm(cat_cols):
    trans = OneHotEncoder()
    res = trans.fit_transform(data[[col]]).toarray()
    resdf = pd.DataFrame(res).rename(columns=lambda x: f'{col}_{int(x)}').iloc[:, 1:].astype('int')
    data = pd.concat([data, resdf], axis=1)
    data = data.drop(columns=[col])

Started cat_cols


100%|██████████| 25/25 [02:46<00:00,  6.67s/it]


In [11]:
data.shape

(200000, 6114)

###  Разделим выборку на тренировочную и тестовую

P.S. в задачах классификации, как и в задаче регрессии, можно использовать технологию Кросс-Валидации. Например, по одному из двух следующих сценариев:

1) Отделить валидацию и тест, произвести подбор лучшей модели с помощью `K-Fold` на валидации, финально обучить выбранную модель на всей валидации и замерить качество на заранее отложенном финальном тесте!

2) Всю выборку назвать валидационной и на ней применить `K-Fold` без финального замера.

В этой домашней работе попросим Вас быть еще проще! :)
Реализуем просто технологию отложенной выборки в пропорции 3:1

In [12]:
from sklearn.model_selection import train_test_split 

X = data.drop(columns=['HasDetections'])
y = data['HasDetections']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    shuffle=True,
                                                    random_state=1)

Соберите `Pipeline`, реализовав в нем 2 шага: стандартизация данных через `StandardScaler` и обучение логистической регрессии с помощью `LogisticRegression`, положите результаты в переменную `pipe`, а в классе модели `LogisticRegression` укажите параметр `penalty='none'`

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

### Your code is here
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('LR', LogisticRegression(penalty=None))
])

pipe.fit(X_train, y_train)

c:\Users\e.tarasov\.conda\envs\kc_ml\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('scaler', StandardScaler()),
                ('LR', LogisticRegression(penalty=None))])

Чтобы замерить качество работы такой модели на трейне и на тесте воспользуемся функцией `cross_validate`

Вопрос: что передавать ей в параметр cv? Ведь мы уже разделили нашу выборку на трейн и тест (имеем всего 1 fold). Для этого можно просто передать список от кортежа, содержащего индексы тренировочных и тестовых объектов.

In [14]:
from sklearn.model_selection import cross_validate
import datetime

custom_cv = [(X_train.index.to_list(), X_test.index.to_list())]

begin_time = datetime.datetime.now()

cv_result_pipe = cross_validate(pipe, X, y, scoring='accuracy',
                                cv=custom_cv, return_train_score=True)


print(f"Accuracy на трейне: {np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_result_pipe['test_score']).round(3)}")

print(f"Время работы алгоритма: {datetime.datetime.now() - begin_time}")

c:\Users\e.tarasov\.conda\envs\kc_ml\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy на трейне: 0.651
Accuracy на тесте: 0.623
Время работы алгоритма: 0:03:57.602874


Что можете сказать про время работы алгоритма?

Очевидно, оно достаточно большие. Уж тем более для линейных моделей.

Такое как раз-таки происходит из-за того, что количество фичей, который мы передали нашей модели - гигантское! Классу требуется много времени и памяти, чтобы обработать датасет.

Поэтому те колонки, в которых количество уникальных категорий превышает какое-то адекватное число, следует кодировать иначе, нежели с помощью технологии `One-Hot-Encoding`.

Теперь вы верите, что более умные кодировки зачастую прям необходимы! Раньше мы этот факт не демонстрировали!

Дело еще вот в чем: в классе `LogisticRegression`, как и, например, `Lasso`, есть параметр, ограничивающий максимальное количество итераций во время обучения модели. Так, если данных много и итераций тоже ожидается большое число, найденная разделяющая гиперплоскость может оказаться не самой лучшей, так как наш алгоритм (будь то градиентный спуск или любой иной) просто 'не доползет'. 

#### Задание 2

Теперь попробуем другой метод кодирования категориальных колонок, а именно счётчики.
Построем ту же модель и на том же разделении, просто заново иначе переобработаем датасет. 

Для тех категориальных признаков, у которых количество уникальных значений в колоночках больше 5, применим `MeanTargetEncoding`.

Для остальных оставим любимый `OneHotEncoding` (как делали на практике и в предыдущем уроке).

In [15]:
### Your code is here
for col in tqdm(data_2.columns):
    if data_2[col].dtype == 'object':
        if len(data_2[col].unique()) > 5:
            mean_values = data_2.loc[:,[col, 'HasDetections']].\
                groupby(col)['HasDetections'].\
                    mean().reset_index().\
                        rename(columns={'HasDetections': f'{col}_enc'})
            data_2 = data_2.merge(mean_values, how='left', on=col).drop(columns=[col])
        else:
            trans = OneHotEncoder(drop='first')
            res = trans.fit_transform(data_2[[col]]).toarray()
            resdf = pd.DataFrame(res).rename(columns=lambda x: f'{col}_{int(x)}').iloc[:, 1:].astype('int')
            data_2 = pd.concat([data_2, resdf], axis=1)
            data_2 = data_2.drop(columns=[col])

  0%|          | 0/73 [00:00<?, ?it/s]

100%|██████████| 73/73 [00:04<00:00, 16.01it/s]


In [16]:
X_2 = data_2.drop(columns=['HasDetections'])
y_2 = data_2['HasDetections']

Опять обучим модель, пока что без изменений! Скажите, стало ли быстрее? А что с качеством?

In [17]:
### Your code is here
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X_2, y_2,
                                                    test_size=0.25,
                                                    shuffle=True,
                                                    random_state=1)

### Your code is here
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('LR', LogisticRegression(penalty=None))
])

pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('LR', LogisticRegression(penalty=None))])

In [18]:
custom_cv = [(X_train.index.to_list(), X_test.index.to_list())]

begin_time = datetime.datetime.now()

cv_result_pipe = cross_validate(pipe, X_2, y_2, scoring='accuracy',
                                cv=custom_cv, return_train_score=True)


print(f"Accuracy на трейне: {np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_result_pipe['test_score']).round(3)}")

print(f"Время работы алгоритма: {datetime.datetime.now() - begin_time}")

Accuracy на трейне: 0.638
Accuracy на тесте: 0.638
Время работы алгоритма: 0:00:03.358321


#### Задание 3: Регуляризация

Как и в моделях регрессии, решая задачу классификации, можем штрафовать минимизируемый функционал за большие веса, добавив к нему L1 или L2 норму весов (все как раньше!).

Для этого в изначальном классе `LogisticRegression` изменить параметр `penalty` на l1 или l2 соответственно. Выберите второй вариант! Можно воспользоваться методом `set_params` и применить его к `pipe`.

In [19]:
### Your code is here
### Your code is here
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('LR', LogisticRegression(penalty='l2'))
])

# pipe.fit(X_train, y_train)

custom_cv = [(X_train.index.to_list(), X_test.index.to_list())]

begin_time = datetime.datetime.now()

cv_result_pipe = cross_validate(pipe, X_2, y_2, scoring='accuracy',
                                cv=custom_cv, return_train_score=True)


print(f"Accuracy на трейне: {np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_result_pipe['test_score']).round(3)}")

print(f"Время работы алгоритма: {datetime.datetime.now() - begin_time}")

Accuracy на трейне: 0.638
Accuracy на тесте: 0.638
Время работы алгоритма: 0:00:03.280562


Теперь наша модель будет строить логистическую регрессию с L2 регуляризатором! Помним, что у регуляризируемых моделей есть гиперпараметр, контролирующий силу регуляризации, который выбирается ДО запуска метода fit, то есть заранее, когда модель еще не обучена. 

Конечно же, выбор этого параметра влияет итоговые результаты. Хочется поперебирать различные параметры регуляризации и найти такой, при котором качество на тесте окажется лучшим! 

Сгенерируем массив гиперпараметра, которые планируем перебрать:

In [20]:
alphas = np.linspace(0.01, 100, 100).tolist()
alphas

[0.01,
 1.02,
 2.03,
 3.04,
 4.05,
 5.06,
 6.07,
 7.08,
 8.09,
 9.1,
 10.11,
 11.12,
 12.13,
 13.14,
 14.15,
 15.16,
 16.17,
 17.180000000000003,
 18.19,
 19.200000000000003,
 20.21,
 21.220000000000002,
 22.23,
 23.240000000000002,
 24.250000000000004,
 25.26,
 26.270000000000003,
 27.28,
 28.290000000000003,
 29.3,
 30.310000000000002,
 31.32,
 32.33,
 33.339999999999996,
 34.35,
 35.36,
 36.37,
 37.379999999999995,
 38.39,
 39.4,
 40.41,
 41.42,
 42.43,
 43.44,
 44.449999999999996,
 45.46,
 46.47,
 47.48,
 48.49,
 49.5,
 50.51,
 51.519999999999996,
 52.53,
 53.54,
 54.55,
 55.559999999999995,
 56.57,
 57.58,
 58.589999999999996,
 59.6,
 60.61,
 61.62,
 62.629999999999995,
 63.64,
 64.65,
 65.66000000000001,
 66.67,
 67.68,
 68.69000000000001,
 69.7,
 70.71000000000001,
 71.72,
 72.73,
 73.74000000000001,
 74.75,
 75.76,
 76.77000000000001,
 77.78,
 78.79,
 79.80000000000001,
 80.81,
 81.82000000000001,
 82.83000000000001,
 83.84,
 84.85000000000001,
 85.86,
 86.87,
 87.8800000000000

Чтобы отобрать среди данного массива гиперпараметров лучший, воспользуйтесь конструкцией `GridSearchCV` из `sklearn`

In [25]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'LR__C': alphas
}

### Your code is here
clf = GridSearchCV(
    pipe,
    param_grid = param_grid,
    cv=custom_cv
)

In [26]:
clf.fit(X_2, y_2)

GridSearchCV(cv=[([76034, 51652, 138222, 115575, 45249, 179692, 182698, 176974,
                   110306, 46620, 175394, 76233, 50519, 22013, 78414, 53567,
                   123438, 33583, 80847, 173139, 96573, 136658, 120025, 75269,
                   157138, 40325, 34982, 27938, 163628, 63394, ...],
                  [49673, 171551, 5506, 38370, 36930, 167783, 67487, 101486,
                   131774, 48246, 147174, 72470, 20386, 6389, 171910, 105548,
                   180639, 155152, 140357, 177097, 122...
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('LR', LogisticRegression())]),
             param_grid={'LR__C': [0.01, 1.02, 2.03, 3.04, 4.05, 5.06, 6.07,
                                   7.08, 8.09, 9.1, 10.11, 11.12, 12.13, 13.14,
                                   14.15, 15.16, 16.17, 17.180000000000003,
                                   18.19, 19.200000000000003, 20.21,
                                   21.220000000000002, 22.23,
                                   23.240000000000002, 24.250000000000004,
                                   25.26, 26.270000000000003, 27.28,
                                   28.290000000000003, 29.3, ...]})

In [30]:
clf.best_params_, round(clf.best_score_, 3)

({'LR__C': 5.06}, 0.638)

#### Задание 4: Бонус

Как вы знаете, подбор признаков является одной из самых важных частей машинного обучения. Сейчас вы попробуете на основе имеющихся признаков сгенерировать новые. В качестве новых признаков будем использовать мономы 2 степени. Можно использовать регуляризацию различного рода, выбор энкодера тоже за вами. Ваша задача - добиться качества `0.65` на тестовой выборке

In [ ]:
### Your code is here

